# Time Period: 20150331-20150630

In [1]:
import pandas as pd
import glob
import os
import numpy as np
from sklearn import linear_model
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
AShareEODDerivativeIndicator = pd.read_csv("/Users/yiliwang/Documents/ACR Internship/Supplementary datasets/EODDerivativeIndicator.csv", header=0)

### X1

In [3]:
BEME_ratio_df_2015 = pd.read_csv("/Users/yiliwang/Documents/ACR Internship/Three_Factor_Model/2015_Q1/BEME_ratio_df_2015.csv", header=0)

In [4]:
norm_BEME_ratio_list = list(BEME_ratio_df_2015['norm_BEME_ratio'])

### X2: Last Quarter Return: 2015 Q1

In [5]:
os.chdir('/Users/yiliwang/Documents/ACR Internship/data/')
files = [file for file in os.listdir( os.curdir ) if file.endswith(".csv")]
# Store files to dictionary using their own name
frames = {}
for each_file in files:
    frames[os.path.splitext(each_file)[0]] = pd.read_csv(each_file)

In [6]:
start_close = [None]*len(BEME_ratio_df_2015['WINDCODE'])
end_close = [None]*len(BEME_ratio_df_2015['WINDCODE'])

In [7]:
for i in range(len(BEME_ratio_df_2015['WINDCODE'])):
    company = BEME_ratio_df_2015['WINDCODE'][i]
    start_close[i] =float(frames[company][frames[company]['TRADE_DT'] == 20141231]['S_DQ_ADJCLOSE'])
    end_close[i] = float(frames[company][frames[company]['TRADE_DT'] == 20150331]['S_DQ_ADJCLOSE'])

In [8]:
com_qtr_return = [None]*len(BEME_ratio_df_2015['WINDCODE'])

In [9]:
for i in range(len(BEME_ratio_df_2015['WINDCODE'])):
    com_qtr_return[i] = (end_close[i] - start_close[i])/start_close[i]

In [12]:
csi300_IndexData= pd.read_csv("/Users/yiliwang/Documents/ACR Internship/csi_300_IndexData_daily_.csv", header=0)

In [15]:
index_start = float(csi300_IndexData[csi300_IndexData['date'] == '2014-12-31']['last_price']) # index price on start day
index_end = float(csi300_IndexData[csi300_IndexData['date'] == '2015-03-31']['last_price'].mean())  # index price on end day

In [16]:
index_qtr_return = (index_end - index_start)/index_start # calculate the index quarterly return

In [17]:
print index_qtr_return

0.146766492994


In [18]:
diff_return_com_csi300 = [None]*len(BEME_ratio_df_2015['WINDCODE'])

In [19]:
for i in range(len(BEME_ratio_df_2015['WINDCODE'])):
    diff_return_com_csi300[i] = com_qtr_return[i] - index_qtr_return

In [20]:
Weighted_Return = pd.DataFrame({'Company':BEME_ratio_df_2015['WINDCODE'],'diff_csi300_LAST': diff_return_com_csi300})

In [21]:
# Normalize Function
def normalize_list(input_list):
    empty_list = [None]*len(input_list)
    for i in range(len(input_list)):
        empty_list[i] = (input_list[i]-input_list.mean())/input_list.std()
    return empty_list

In [22]:
Weighted_Return['norm_diff_csi300_LAST'] = normalize_list(Weighted_Return['diff_csi300_LAST'])

In [23]:
Weighted_Return.to_csv("LAST_Weighted_Return_2015_Q2.csv")

### X3

In [24]:
# Extract trading dates of certain period 
dates_trade = frames['000001.SZ']['TRADE_DT'].unique()
dates_trade.sort() # sort the date

start_dateind = list(dates_trade).index(20150331) # index of start date
end_dateind = list(dates_trade).index(20150630) # index of end date

In [25]:
target_dates = dates_trade[start_dateind:end_dateind+1]

In [26]:
target_dates

array([20150331, 20150401, 20150402, 20150403, 20150407, 20150408,
       20150409, 20150410, 20150413, 20150414, 20150415, 20150416,
       20150417, 20150420, 20150421, 20150422, 20150423, 20150424,
       20150427, 20150428, 20150429, 20150430, 20150504, 20150505,
       20150506, 20150507, 20150508, 20150511, 20150512, 20150513,
       20150514, 20150515, 20150518, 20150519, 20150520, 20150521,
       20150522, 20150525, 20150526, 20150527, 20150528, 20150529,
       20150601, 20150602, 20150603, 20150604, 20150605, 20150608,
       20150609, 20150610, 20150611, 20150612, 20150615, 20150616,
       20150617, 20150618, 20150619, 20150623, 20150624, 20150625,
       20150626, 20150629, 20150630])

In [27]:
def find_FAST_mean(company): # For a given company, calculate mean of FLOAT_A_SHR_TODAY *close price (unadjusted) on trading days
    FAST_product = [None]*len(target_dates)
    
    for i in range(len(target_dates)):
        close_p = frames[company][frames[company]['TRADE_DT'] == target_dates[i]]['S_DQ_CLOSE'].values # get close price on a particular date
        
        # dataframe of the company
        df_company = AShareEODDerivativeIndicator[AShareEODDerivativeIndicator['S_INFO_WINDCODE'] ==company]
        FAST_day = float(df_company[df_company['TRADE_DT']==target_dates[i]]['FLOAT_A_SHR_TODAY']) #FLOAT_A_SHR_TODAY of a particular day
        
        FAST_product[i] = FAST_day*close_p
        
    FAST_mean = np.mean(FAST_product)
    return FAST_mean

In [28]:
FAST_mean_list = [find_FAST_mean(company) for company in BEME_ratio_df_2015['WINDCODE']]

In [30]:
FAST_mean_df = pd.DataFrame({'Company':BEME_ratio_df_2015['WINDCODE'],'FAST_mean':FAST_mean_list})

In [31]:
FAST_mean_df['FAST_mean_norm'] = normalize_list(FAST_mean_df['FAST_mean'])

In [63]:
FAST_mean_df[:10]

,Company,FAST_mean,FAST_mean_norm
0,000157.SZ,5.216800e+06,-0.222568
1,600887.SH,1.117917e+07,0.091662
2,600276.SH,8.482944e+06,-0.050435
3,600009.SH,3.239866e+06,-0.326756
4,601328.SH,2.382621e+07,0.758187
5,600150.SH,7.731130e+06,-0.090057
6,000728.SZ,7.085288e+06,-0.124094
7,600036.SH,3.744514e+07,1.475932
8,600660.SH,3.253200e+06,-0.326053
9,601998.SH,2.489464e+07,0.814495


In [34]:
FAST_mean_df.to_csv("FAST_mean_2015_Q2.csv") # save it as csv

### Y

In [35]:
# Obatin Close Price on starting and end dates for current qtr
start_close_c = [None]*len(BEME_ratio_df_2015['WINDCODE'])
end_close_c = [None]*len(BEME_ratio_df_2015['WINDCODE'])

In [36]:
for i in range(len(BEME_ratio_df_2015['WINDCODE'])):
    company = BEME_ratio_df_2015['WINDCODE'][i]
    start_close_c[i] =float(frames[company][frames[company]['TRADE_DT'] == 20150331]['S_DQ_ADJCLOSE'])
    end_close_c[i] = float(frames[company][frames[company]['TRADE_DT'] ==20150630]['S_DQ_ADJCLOSE'])

In [37]:
com_qtr_return_CURRENT = [None]*len(BEME_ratio_df_2015['WINDCODE']) 

In [38]:
for i in range(len(BEME_ratio_df_2015['WINDCODE'])):
    com_qtr_return_CURRENT[i] = (end_close_c[i] - start_close_c[i])/start_close_c[i]

In [45]:
index_start_c = float(csi300_IndexData[csi300_IndexData['date'] == '2015-03-31']['last_price'].mean()) 
index_end_c = float(csi300_IndexData[csi300_IndexData['date'] == '2015-06-30']['last_price'].mean()) 

In [46]:
index_qtr_return_CURRENT = (index_end_c - index_start_c)/index_start_c # calculate the index quarterly return

In [47]:
index_qtr_return_CURRENT

0.10393219259809501

In [48]:
diff_return_com_csi300_CURRENT = [None]*len(BEME_ratio_df_2015['WINDCODE'])

In [49]:
for i in range(len(BEME_ratio_df_2015['WINDCODE'])):
    diff_return_com_csi300_CURRENT[i] = com_qtr_return_CURRENT[i] - index_qtr_return_CURRENT

In [50]:
diff_return_com_csi300_CURRENT[0:10]

[0.003992233118909172,
 0.1442612287570138,
 0.15401130109617067,
 0.20917145451160657,
 0.23366099164045662,
 0.04019302762144679,
 0.007294852477030156,
 0.09829759990732466,
 -0.21358641995956074,
 -0.0557006813440758]

In [53]:
Y_2015_Q2 = pd.DataFrame({'Company':BEME_ratio_df_2015['WINDCODE'],'Y':diff_return_com_csi300_CURRENT})

In [54]:
Y_2015_Q2.to_csv("Y_2015_Q2.csv")

In [55]:
TFM_2015_Q2 = pd.DataFrame({'X_1':norm_BEME_ratio_list,'X_2':Weighted_Return['norm_diff_csi300_LAST'], 'X_3': FAST_mean_df['FAST_mean_norm'], 'Y':diff_return_com_csi300_CURRENT})

In [62]:
TFM_2015_Q2[:10]

,X_1,X_2,X_3,Y
0,1.492577,-0.775226,-0.222568,0.003992
1,-0.716115,-0.668291,0.091662,0.144261
2,-0.981127,-0.268099,-0.050435,0.154011
3,0.368114,-0.267052,-0.326756,0.209171
4,2.030339,-1.029491,0.758187,0.233661
5,0.009483,-0.290717,-0.090057,0.040193
6,-0.458558,-0.610429,-0.124094,0.007295
7,1.308653,-1.033202,1.475932,0.098298
8,-0.185719,0.120613,-0.326053,-0.213586
9,1.127622,-1.123648,0.814495,-0.055701


In [57]:
TFM_2015_Q2.to_csv("TFM_2015_Q2.csv")

In [58]:
def outlier_index(list_name, std_num):
    mean = list_name.mean() # mean
    std = np.std(list_name, axis=0) # standard deviation
    
    upper_b = mean+std_num*std
    lower_b = mean-std_num*std
    
    ind_upper = [i for i in range(len(list_name)) if list_name[i] > upper_b]
    ind_lower = [i for i in range(len(list_name)) if list_name[i] < lower_b]
    outlier_index = ind_upper+ind_lower
    
    return outlier_index

In [59]:
outlier_all =outlier_index(TFM_2015_Q2['X_1'], 5) + outlier_index(TFM_2015_Q2['X_2'], 5)+ outlier_index(TFM_2015_Q2['X_3'], 5)+ outlier_index(TFM_2015_Q2['Y'], 5)

In [60]:
TFM_2015_Q2_NO = TFM_2015_Q2.drop(TFM_2015_Q2.index[outlier_all])

In [61]:
TFM_2015_Q2_NO.to_csv("TFM_2015_Q2_NO.csv")

# ************* Result DataFrame: TFM_2015_Q2_NO.csv *************